In [1]:
mutable struct NeuralNetwork
    weights::Array{Array{Number,2},1}
    biases::Array{Array{Number,1},1}
end

In [2]:
# Function takes an empty network and a tuple of layers
# to create bias vectors and weight matricies of 
# appropriate sizes

function createNetwork!(network::NeuralNetwork,layers::Tuple)  
    for i in 1:length(layers)-1
        weightMatrix = randn(layers[i+1],layers[i])
        push!(network.weights,weightMatrix)
        biasVector = randn(layers[i+1])
        push!(network.biases,biasVector)
    end
 end

createNetwork! (generic function with 1 method)

In [3]:
sigmoid(x::Real) = 1/(1 + exp(-x))
sigmoidPrime(x::Real) = sigmoid(x)*(1-sigmoid(x))

sigmoidPrime (generic function with 1 method)

In [21]:
# Function takes a network and feeds an input vector through
# to return lists of activations and z = (W*a + b)

function feedForward(network::NeuralNetwork, input::Array{Real,1})
    
    a = input
    activations = [a]
    z::Array{Real,1} = []
    zList::Array{Array{Real,1},1} = []
    
    for (W,b) in zip(network.weights, network.biases)
        z = W*a + b
        push!(zList, z)
        a = sigmoid.(z)
        push!(activations, a)
    end
    
    return activations, zList
end


function feedForward(network::NeuralNetwork, input::Array{Float64,1})
    
    a = input
    activations = [a]
    z::Array{Real,1} = []
    zList::Array{Array{Real,1},1} = []
    
    for (W,b) in zip(network.weights, network.biases)
        z = W*a + b
        push!(zList, z)
        a = sigmoid.(z)
        push!(activations, a)
    end
    
    return activations, zList
end

function feedForward(network::NeuralNetwork, input::Array)
    
    a = input
    activations = [a]
    z::Array{Real,1} = []
    zList::Array{Array{Real,1},1} = []
    
    for (W,b) in zip(network.weights, network.biases)
        z = W*a + b
        push!(zList, z)
        a = sigmoid.(z)
        push!(activations, a)
    end
    
    return activations, zList
end

feedForward (generic function with 3 methods)

In [5]:
# function takes an image label number and creates a 10x1 vector 
# with a 1 in the position of the number
 
function labelToVector(label::Number)::Array
    labelVector = zeros(10)
    labelVector[label + 1] = 1
    return labelVector
end

labelToVector (generic function with 1 method)

In [6]:
using LinearAlgebra

# Function takes a network, input, and label to compute gradients
# for the network's weights and biases and returns these gradients 
# in lists

function backPropagation(network::NeuralNetwork, input::Array{Number,1}, label::Array{Number,1})
    nabla_w::Array{Array{Number,2},1} = [] # Array to hold weight gradients
    nabla_b::Array{Array{Number,1},1} = [] # Array to hold bias gradients
    aList, zList = feedForward(network, input)
    delta = (aList[end] - label) .* sigmoidPrime.(zList[end])
    pushfirst!(nabla_b, delta)
    wDelta = delta * aList[end - 1]'
    pushfirst!(nabla_w, wDelta)
    
    for i in 0:length(network.weights)-2
        delta = (net.weights[end - i]' * delta) .* sigmoidPrime.(zList[end - i - 1])
        pushfirst!(nabla_b, delta)
        wDelta = delta * aList[end - i - 2]'
        pushfirst!(nabla_w, wDelta)
    end
    
    return nabla_b, nabla_w
end


function backPropagation(network::NeuralNetwork, input::Array{Real,1}, label::Array{Real,1})
    nabla_w::Array{Array{Number,2},1} = [] # Array to hold weight gradients
    nabla_b::Array{Array{Number,1},1} = [] # Array to hold bias gradients
    aList, zList = feedForward(network, input)
    delta = (aList[end] - label) .* sigmoidPrime.(zList[end])
    pushfirst!(nabla_b, delta)
    wDelta = delta * aList[end - 1]'
    pushfirst!(nabla_w, wDelta)
    
    for i in 0:length(network.weights)-2
        delta = (net.weights[end - i]' * delta) .* sigmoidPrime.(zList[end - i - 1])
        pushfirst!(nabla_b, delta)
        wDelta = delta * aList[end - i - 2]'
        pushfirst!(nabla_w, wDelta)
    end
    
    return nabla_b, nabla_w
end


function backPropagation(network::NeuralNetwork, input::Array, label::Array)
    nabla_w::Array{Array{Number,2},1} = [] # Array to hold weight gradients
    nabla_b::Array{Array{Number,1},1} = [] # Array to hold bias gradients
    aList, zList = feedForward(network, input)
    delta = (aList[end] - label) .* sigmoidPrime.(zList[end])
    pushfirst!(nabla_b, delta)
    wDelta = delta * aList[end - 1]'
    pushfirst!(nabla_w, wDelta)
    
    for i in 0:length(network.weights)-2
        delta = (net.weights[end - i]' * delta) .* sigmoidPrime.(zList[end - i - 1])
        pushfirst!(nabla_b, delta)
        wDelta = delta * aList[end - i - 2]'
        pushfirst!(nabla_w, wDelta)
    end
    
    return nabla_b, nabla_w
end

function backPropagation(network::NeuralNetwork, input::Array, label::Number)
    label = labelToVector(label)
    nabla_w::Array{Array{Number,2},1} = [] # Array to hold weight gradients
    nabla_b::Array{Array{Number,1},1} = [] # Array to hold bias gradients
    aList, zList = feedForward(network, input)
    delta = (aList[end] - label) .* sigmoidPrime.(zList[end])
    pushfirst!(nabla_b, delta)
    wDelta = delta * aList[end - 1]'
    pushfirst!(nabla_w, wDelta)
    
    for i in 0:length(network.weights)-2
        delta = (net.weights[end - i]' * delta) .* sigmoidPrime.(zList[end - i - 1])
        pushfirst!(nabla_b, delta)
        wDelta = delta * aList[end - i - 2]'
        pushfirst!(nabla_w, wDelta)
    end
    
    return nabla_b, nabla_w
end

backPropagation (generic function with 4 methods)

In [15]:
# Function takes a network, minibatch, and learning rate (eta)
# to update the networks weights and biases by applying 
# gradient decent using backpropagation to a single minibatch

function updateMiniBatch!(network::NeuralNetwork, miniBatch::Array, eta::Number)
    m = length(miniBatch)
    for (input, label) in miniBatch
        deltaB, deltaW = backPropagation(network, input, label)
        network.weights = [W-(eta/m)*nW for (W, nW) in zip(network.weights,deltaW)]
        network.biases = [b-(eta/m)*nb for (b, nb) in zip(network.biases,deltaB)]
    end
end

function updateMiniBatch!(network::NeuralNetwork, miniBatch::Base.Iterators.Zip2{Array{Any,1},Array{Int64,1}}, eta::Float64)
    m = length(miniBatch)
    for (input, label) in miniBatch
        deltaB, deltaW = backPropagation(network, input, label)
        network.weights = [W-(eta/m)*nW for (W, nW) in zip(network.weights,deltaW)]
        network.biases = [b-(eta/m)*nb for (b, nb) in zip(network.biases,deltaB)]
    end
end

updateMiniBatch! (generic function with 2 methods)

In [8]:
a = []
b = []
net = NeuralNetwork(a,b)

NeuralNetwork(Array{Number,2}[], Array{Number,1}[])

In [9]:
createNetwork!(net,(784,200,10))

In [10]:
@show net.weights[2]

net.weights[2] = Number[0.182016 0.401521 -0.144641 -0.272758 0.852942 0.315119 -2.43303 -1.23757 -1.38621 -0.399611 0.757027 -0.176146 -0.23273 0.409813 -0.535032 -0.288317 -1.34688 0.212252 2.22438 -0.944614 0.755786 -1.15988 0.456596 -1.06598 0.464317 -1.84364 1.60636 -1.47028 0.97072 0.317286 0.444582 -0.468924 -0.69745 -1.25033 0.00113784 -0.199591 -0.307802 0.482973 1.35875 1.81312 -0.809124 0.124592 -0.456412 2.11043 -0.415014 -0.709311 -1.48382 -0.965337 0.331794 1.06699 0.700051 0.337754 0.481906 -0.853025 1.84972 -0.531228 -1.22721 1.3225 -0.0617893 0.296143 -2.28818 -2.31865 1.42092 0.22793 -0.109339 0.198952 0.551391 0.0829422 0.683367 1.51461 -1.50606 -0.0614496 1.60142 -0.776964 -0.377887 -0.935962 -0.908844 -0.0873855 -0.838814 0.186442 -0.216925 1.41853 -1.39776 0.674235 0.969322 -0.202031 1.86509 0.516469 0.544624 1.42014 0.564784 0.595717 0.276395 1.29563 -0.211574 0.157918 1.12962 -0.691763 1.09206 -1.94625 0.177151 0.212148 0.420582 0.824384 -0.235138 0.664799 0.496

10×200 Array{Number,2}:
  0.182016     0.401521  -0.144641  …   1.24639   -0.409293   0.424506
 -0.119076    -0.43071   -1.29235       0.766846   0.69674    0.581029
  0.18512      0.402763   0.18014      -0.430791  -0.746894  -0.608519
  0.557546     0.938203   1.09739       2.42963   -1.35817   -0.228297
  0.00783157   0.294111   2.46051       1.22279   -1.00447   -0.465815
 -0.408201     0.833664   0.873638  …  -1.52272   -1.26134    0.790957
 -0.855984     0.336376  -0.559752      0.416991  -0.524418  -0.321142
  0.034741     0.846458  -1.01629      -1.81761    1.479      0.634244
 -1.05969      0.314106   1.44681      -1.61564    0.904367  -0.275499
  0.889561    -0.237815  -0.627666      0.105258  -0.386529  -2.09783 

In [11]:
@show net.biases[2]

net.biases[2] = Number[-0.803328, -1.93766, -0.659939, -1.40606, 1.41403, 0.241304, -0.237793, -0.537005, 0.484212, -0.637941]


10-element Array{Number,1}:
 -0.8033277453157116
 -1.937662274136535 
 -0.6599389514794575
 -1.406062153454738 
  1.41403382405965  
  0.2413039018731661
 -0.2377933261229531
 -0.537004776178172 
  0.4842115960020044
 -0.6379414671874847

In [12]:
using MLDatasets
train_x, train_y = MNIST.traindata(1:5)

(FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [5, 0, 4, 1, 9])

In [13]:
trainVectors = []
for i in 1:size(train_x,3)
    push!(trainVectors,vec(train_x[:, :, i]))
end

data = zip(trainVectors,train_y)

Base.Iterators.Zip2{Array{Any,1},Array{Int64,1}}(Any[FixedPointNumbers.Normed{UInt8,8}[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], FixedPointNumbers.Normed{UInt8,8}[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], FixedPointNumbers.Normed{UInt8,8}[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], FixedPointNumbers.Normed{UInt8,8}[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], FixedPointNumbers.Normed{UInt8,8}[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [5, 0, 4, 1, 9])

In [22]:
updateMiniBatch!(net, data, 0.1)

In [23]:
@show net.weights[2]

net.weights[2] = Number[0.180877 0.400388 -0.144641 -0.273167 0.852936 0.313987 -2.43303 -1.2376 -1.38735 -0.399617 0.756479 -0.177278 -0.232744 0.409603 -0.53609 -0.288333 -1.34688 0.212249 2.2233 -0.945725 0.755786 -1.15988 0.455729 -1.06712 0.463207 -1.84444 1.60636 -1.47137 0.969916 0.316148 0.443465 -0.469155 -0.697452 -1.25033 -3.47082e-7 -0.199591 -0.308363 0.48184 1.358 1.81205 -0.810187 0.124082 -0.456427 2.11038 -0.416014 -0.710439 -1.48496 -0.965743 0.331794 1.06585 0.699057 0.336737 0.480768 -0.853025 1.84859 -0.532367 -1.22835 1.32248 -0.0617953 0.296143 -2.28928 -2.31978 1.42092 0.226793 -0.110473 0.197819 0.550912 0.08292 0.682233 1.51458 -1.50719 -0.0614533 1.60138 -0.778102 -0.379025 -0.93598 -0.908972 -0.0873916 -0.838814 0.186436 -0.218064 1.41846 -1.39781 0.673103 0.96819 -0.203152 1.86414 0.515332 0.544621 1.41901 0.564731 0.594585 0.275257 1.29562 -0.212685 0.157918 1.12962 -0.691767 1.09126 -1.94738 0.176627 0.211739 0.419457 0.824322 -0.235144 0.664772 0.495602 

10×200 Array{Number,2}:
  0.180877     0.400388  -0.144641  …   1.24623   -0.409293   0.423368
 -0.127256    -0.433871  -1.29689       0.7668     0.691062   0.573806
  0.185101     0.402745   0.180121     -0.430791  -0.746913  -0.608533
  0.557543     0.938201   1.09739       2.42963   -1.35817   -0.2283  
  0.00721269   0.293601   2.45958       1.2228    -1.00487   -0.466693
 -0.408321     0.833589   0.87355   …  -1.52272   -1.26135    0.790802
 -0.856267     0.336002  -0.560266      0.416987  -0.524466  -0.32165 
  0.0341174    0.846061  -1.01668      -1.81761    1.47862    0.633721
 -1.0597       0.314095   1.4468       -1.61564    0.904365  -0.275513
  0.884803    -0.24265   -0.633675      0.105235  -0.389963  -2.10311 